# ⚡️Summary ⚡️

This notebook came from learning about a re-emergence of an statistical methodology to predict and transform data.  Symbolic regression is a machine learning technique that aims to identify an underlying mathematical expressions that best describes relationship in data. 
Symbolic Regression can therefore:
* Predict data 
* Create new features 

The idea came from a fantastic notebook from [Jano123](https://www.kaggle.com/code/jano123/sr-with-inequalities-simple-model) who uses symbolic regression to predict a classification problem\
In this notebook we will look to **Generate Features** using Symbolic Transformation \
All symbolic regression libraries can be found on the [gplearn website](https://gplearn.readthedocs.io/en/stable/intro.html)

In [1]:
!pip install gplearn

In [2]:
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 

import gc
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

from sklearn.metrics import log_loss, cohen_kappa_score, mean_squared_error, roc_auc_score, r2_score
from sklearn.model_selection import KFold, RepeatedStratifiedKFold,StratifiedKFold, RepeatedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer

import lightgbm as lgb
import catboost as cat
import xgboost as xgb
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.svm import SVR
import shap 


In [3]:
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# parameters 
ADD_DATA = True
TEST_ON_GENERATED_ONLY = True

MADE_COLUMN_DROP = False  ## try set this value to some other year 
DROP_CITY_CODE = True    # if false we run OHE

CV_TEST = False
NUM_FOLDS = 5
NUM_SPLITS = 3

EPOCHS = 5000
SCALING = True
DROP_OUTLIERS = False
SHAP_VALS = True

# Notebook settings
sns.set_style("darkgrid")
pd.set_option('mode.chained_assignment',None)

## Helper functions

In [5]:
def run_model(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    
    lgb_train = lgb.Dataset(X, y)
    lgb_results = lgb.cv(lgb_params, lgb_train, num_boost_round=EPOCHS, nfold=5, metrics='rmse', stratified=False)
    score_lgb = pd.DataFrame(lgb_results)["rmse-mean"].min()
    print("LIGHTGBM RSME",score_lgb)
    
    dtrain = xgb.DMatrix(X, label = y)
    xgb_results=  xgb.cv(xgb_params, dtrain,  num_boost_round=EPOCHS, early_stopping_rounds=30, metrics = {"rmse"}, nfold=5)
    score_xgb = xgb_results["test-rmse-mean"].min()
    print("XGBOOST RSME",score_xgb)
    
    return score_lgb, score_xgb


## 💾 Load Data 💾
Data taken from TPS May 2022 competition 

In [6]:
target = "Strength" #Target column that we will be predicting, this is here for quick reference 

df_train = pd.read_csv("/kaggle/input/playground-series-s3e9/train.csv", index_col = 0)
df_test = pd.read_csv("/kaggle/input/playground-series-s3e9/test.csv", index_col = 0)
sub = pd.read_csv("/kaggle/input/playground-series-s3e9/sample_submission.csv",index_col = 0)
df_train

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength
id,,,,,,,,,
0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38
1,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52
2,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96
3,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05
4,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19
...,...,...,...,...,...,...,...,...,...
5402,446.0,24.0,79.0,162.0,11.6,967.0,712.0,3,15.42
5403,350.0,0.0,0.0,203.0,0.0,974.0,775.0,180,49.20
5404,295.8,0.0,0.0,185.7,0.0,1076.2,759.3,28,39.30


In [7]:
if ADD_DATA:
    add_data = pd.read_csv('/kaggle/input/predict-concrete-strength/ConcreteStrengthData.csv')
    
    print("length of additional data", len(add_data))
    print("length of original data", len(df_train))
    df_train['is_generated'] = 1
    df_test['is_generated'] = 1
    add_data['is_generated'] = 0
    add_data.columns =df_train.columns

    df_train = pd.concat([df_train, add_data],axis=0, ignore_index=True)
df_train

length of additional data 1030
length of original data 5407


,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength,is_generated
0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38,1
1,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52,1
2,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96,1
3,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05,1
4,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19,1
...,...,...,...,...,...,...,...,...,...,...
6432,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28,0
6433,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18,0
6434,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70,0
6435,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77,0


# Baseline Modelling

In [8]:
lgb_params ={'objective': 'regression',# regression_l1, huber, fair, poisson, quantile, mape, gamma, tweedie
             "metric":"rmse", 
             "boosting": "gbdt",#"dart",gbdt
             'n_estimators':EPOCHS,
             'learning_rate':0.01,
            'device':'cpu', 
             'force_col_wise': True,
             'verbose' :0
                   }
xgb_params = { 
    'objective' : "reg:squarederror",
    'eval_metric' : "rmse"
             }

In [9]:
X = df_train.drop([target],axis =1)
y= df_train[target]
score_lgb_base, score_xgb_base = run_model(X,y)

LIGHTGBM RSME 11.432613403660998
XGBOOST RSME 11.508487774570057


# 🎯 Manual Feature Engineering 🎯
We will create a number of manual features as these were identified by some [clever people](https://www.kaggle.com/competitions/playground-series-s3e5/discussion/382698) in the kaggle competition for this dataset 

In [10]:
df_trn = df_train.copy(deep = True)
df_tst = df_test.copy(deep = True)

In [11]:
def additional_feats(df):
    df['Water_Cement'] = df['WaterComponent']/df['CementComponent']
    df['Coarse_Fine'] = df['CoarseAggregateComponent']/df['FineAggregateComponent']
    df['Aggregate'] = df['CoarseAggregateComponent'] + df['FineAggregateComponent']
    df['Aggregate_Cement'] = df['Aggregate']/df['CementComponent']
    df['Slag_Cement'] = df['BlastFurnaceSlag']/df['CementComponent']
    df['Ash_Cement'] = df['FlyAshComponent']/df['CementComponent']
    df['Plastic_Cement'] = df['SuperplasticizerComponent']/df['CementComponent']
    df['Age_Water'] = df['AgeInDays']/df['WaterComponent']
    
    return df
additional_feats(df_trn)
additional_feats(df_tst)

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,is_generated,Water_Cement,Coarse_Fine,Aggregate,Aggregate_Cement,Slag_Cement,Ash_Cement,Plastic_Cement,Age_Water
id,,,,,,,,,,,,,,,,,
5407,166.1,75.4,163.8,173.8,4.6,1007.2,746.6,56,1,1.046358,1.349049,1753.8,10.558700,0.453943,0.986153,0.027694,0.322209
5408,304.0,0.0,0.0,190.0,0.0,998.0,801.0,7,1,0.625000,1.245943,1799.0,5.917763,0.000000,0.000000,0.000000,0.036842
5409,225.0,0.0,0.0,185.0,0.0,1113.0,833.0,28,1,0.822222,1.336134,1946.0,8.648889,0.000000,0.000000,0.000000,0.151351
5410,251.4,0.0,118.3,188.5,6.4,1028.4,757.7,100,1,0.749801,1.357265,1786.1,7.104614,0.000000,0.470565,0.025457,0.530504
5411,144.0,15.0,195.0,176.0,6.0,1021.0,709.0,28,1,1.222222,1.440056,1730.0,12.013889,0.104167,1.354167,0.041667,0.159091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9007,140.0,129.0,100.0,200.0,7.0,864.0,753.0,28,1,1.428571,1.147410,1617.0,11.550000,0.921429,0.714286,0.050000,0.140000
9008,281.0,0.0,0.0,186.0,0.0,1104.0,774.0,28,1,0.661922,1.426357,1878.0,6.683274,0.000000,0.000000,0.000000,0.150538
9009,289.0,133.0,0.0,194.0,7.0,924.0,760.0,28,1,0.671280,1.215789,1684.0,5.826990,0.460208,0.000000,0.024221,0.144330


In [12]:
if ADD_DATA:
    group1_float =df_test.drop("is_generated",axis =1).columns
else: 
    group1_float =df_test.columns
    
def mathematical_feats(df,cols, suffix):
    df[f"sum_{suffix}"] = df[cols].sum(axis = 1)
    df[f"mean_{suffix}"] = df[cols].mean(axis = 1)
    df[f"std_{suffix}"] = df[cols].std(axis = 1)
    df[f"min_{suffix}"] = df[cols].min(axis = 1)
    df[f"max_{suffix}"] = df[cols].max(axis = 1)
    df[f"median_{suffix}"] = df[cols].median(axis = 1)
    df[f"mad_{suffix}"] = df[cols].mad(axis = 1)
    
    df[f"max-min_{suffix}"] = df[cols].max(axis = 1) - df[cols].min(axis = 1)
    df[f"q01_{suffix}"] = df[cols].quantile(q= 0.1, axis =1)
    df[f"q25_{suffix}"] = df[cols].quantile(q= 0.25, axis =1) 
    df[f"q50_{suffix}"] = df[cols].quantile(q= 0.5, axis =1) 
    df[f"q75_{suffix}"] = df[cols].quantile(q= 0.75, axis =1) 
    df[f"q95_{suffix}"] = df[cols].quantile(q= 0.95, axis =1) 
    df[f"q99_{suffix}"] = df[cols].quantile(q= 0.99, axis =1)
    df[f"kurt_{suffix}"] = df[cols].kurt(axis =1) 
    df[f"skew_{suffix}"] = df[cols].skew( axis =1)
    
    return df

# mathematical_feats(df_trn, group1_float, "group1")
# mathematical_feats(df_tst, group1_float, "group1")

# Manual Feats Modelling

In [13]:
X = df_trn.drop([target],axis =1)
y= df_trn[target]
score_lgb_manual, score_xgb_manual = run_model(X,y)

LIGHTGBM RSME 11.473143800961294
XGBOOST RSME 11.608912835769674


# Symbolic Transformation (Feature Generation)
* Symbolic Transformation will run multiple mathematical functions on the dataset and measure the correlation of these new features against the target. 
* Top 100 (we set this under hall_of_fame) correlated features are selected and then only the 10 least-correlated 10 features will be used (we set this under n_components) 

More examples can be found on thie [gplearn website](https://gplearn.readthedocs.io/en/stable/examples.html)

#### Mathematical Functions 
* ‘add’ : addition, arity=2.
* ‘sub’ : subtraction, arity=2.
* ‘mul’ : multiplication, arity=2.
* ‘div’ : division, arity=2.
* ‘sqrt’ : square root, arity=1.
* ‘log’ : log, arity=1.
* ‘abs’ : absolute value, arity=1.
* ‘neg’ : negative, arity=1.
* ‘inv’ : inverse, arity=1.
* ‘max’ : maximum, arity=2.
* ‘min’ : minimum, arity=2.
* ‘sin’ : sine (radians), arity=1.
* ‘cos’ : cosine (radians), arity=1.
* ‘tan’ : tangent (radians), arity=1.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:2px;">
<b><span style="color:blue;font-size:1.2em;">Note:  </span></b>

Running the below seems to produce different results (I assume due to random nature of the generation?)
 * There seems to be two outcomes, one run seems to create a set of features that dont improve the score whereas another run will give significant improvement

In [14]:
from gplearn.genetic import SymbolicTransformer

# drop the text column and target
X = df_train.drop([target],axis =1)
y= df_train[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [15]:
function_set = ['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                'abs', 'neg', 'inv', 'max', 'min']
gp = SymbolicTransformer(generations=40, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0, metric='pearson', 
                         feature_names= X.columns) #option------>metric = 'spearman'
gp.fit(X_train, y_train)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.54         0.139627        4         0.590565         0.602539      1.24m
   1    10.71         0.332182       15          0.60106         0.599006      1.28m
   2     9.41         0.449605       32         0.628139         0.596232      1.31m
   3    11.17         0.448109       18         0.634577         0.612796      1.27m
   4    12.73         0.458762       22         0.640348         0.611524      1.34m
   5    16.77         0.468296       32          0.65713         0.603221      1.33m
   6    18.25         0.496925       42         0.664653         0.592878      1.32m
   7    20.24         0.489318       26         0.663425         0.592707      1.33m
   8    28.57         0.489818       36         0.665631          0.58004  

SymbolicTransformer(feature_names=Index(['CementComponent', 'BlastFurnaceSlag', 'FlyAshComponent',
       'WaterComponent', 'SuperplasticizerComponent',
       'CoarseAggregateComponent', 'FineAggregateComponent', 'AgeInDays',
       'is_generated'],
      dtype='object'),
                    function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    generations=40, max_samples=0.9,
                    parsimony_coefficient=0.0005, population_size=2000,
                    random_state=0, verbose=1)

In [16]:
for function in gp:
    print ("Functions:", function)
    print(f"correlation to target: {function.raw_fitness_:.3f} \n")

Functions: div(log(sub(min(SuperplasticizerComponent, sqrt(WaterComponent)), div(WaterComponent, sqrt(div(log(mul(sqrt(CementComponent), sqrt(AgeInDays))), log(WaterComponent)))))), log(WaterComponent))
correlation to target: 0.678 

Functions: div(log(sub(min(SuperplasticizerComponent, sqrt(WaterComponent)), div(WaterComponent, sqrt(div(log(mul(sqrt(CementComponent), sqrt(AgeInDays))), log(WaterComponent)))))), log(log(sqrt(WaterComponent))))
correlation to target: 0.676 

Functions: div(log(sub(min(SuperplasticizerComponent, sqrt(WaterComponent)), div(WaterComponent, log(log(mul(sqrt(CementComponent), sqrt(AgeInDays))))))), log(WaterComponent))
correlation to target: 0.674 

Functions: div(log(sub(min(SuperplasticizerComponent, sqrt(WaterComponent)), div(WaterComponent, sqrt(div(log(mul(max(sqrt(CementComponent), sqrt(WaterComponent)), sqrt(AgeInDays))), log(WaterComponent)))))), log(WaterComponent))
correlation to target: 0.674 

Functions: div(log(div(log(sub(min(SuperplasticizerCo

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:2px;">
<b><span style="color:blue;font-size:1.2em;">Note:  </span></b>

* If the above functions are only one feature (with no mathemcatical operations applied) we can ignore this run 
    * Try running again with different random_state or try Spearman metric or custom metric (see below)

## Symbolic Feats Transform and Train model (again)
Transform data and join symbolic dataset to Train and Test

In [17]:
sym_trn = gp.transform(X)
sym_tst = gp.transform(df_test)

#create dataframe
sym_trn = pd.DataFrame(sym_trn, columns = [f"sym_{col}" for col in range(sym_tst.shape[1])])
sym_tst = pd.DataFrame(sym_tst, columns = [f"sym_{col}" for col in range(sym_tst.shape[1])])

#join to train and test
new_df_trn = pd.concat([df_trn.reset_index(drop = True),sym_trn],axis =1)
new_df_tst= pd.concat([df_tst.reset_index(drop = True),sym_tst],axis =1)
new_df_trn

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength,is_generated,...,sym_0,sym_1,sym_2,sym_3,sym_4,sym_5,sym_6,sym_7,sym_8,sym_9
0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38,1,...,1.033527,5.623353,0.949326,1.033527,0.006311,0.949326,1.036683,1.036259,0.175206,2.067055
1,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52,1,...,1.015259,5.522881,0.921205,1.011867,0.002883,0.921205,1.016756,1.017000,0.080144,2.030518
2,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96,1,...,1.014296,5.518784,0.921886,1.014296,0.002717,0.921886,1.015654,1.015175,0.074682,2.028591
3,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05,1,...,0.993784,5.402775,0.895950,0.993784,-0.001148,0.895950,0.993210,0.992783,-0.033749,1.987568
4,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19,1,...,1.009094,5.489154,0.914928,1.007112,0.001722,0.914928,1.009955,1.009480,0.047813,2.018189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6432,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28,0,...,1.005171,5.470719,0.907151,1.005171,0.000994,0.907151,1.005692,1.006077,0.026843,2.010343
6433,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18,0,...,1.004410,5.462974,0.905290,1.004410,0.000834,0.905290,1.004848,1.005321,0.023274,2.008819
6434,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70,0,...,1.016740,5.530615,0.923622,1.013729,0.003156,0.923622,1.018364,1.018436,0.088073,2.033481
6435,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77,0,...,1.008484,5.489966,0.911328,1.007285,0.001635,0.911328,1.009352,1.009998,0.043850,2.016969


# Rerun Models
On Symbolic data Only

In [18]:
X = sym_trn
y= df_trn[target]
score_lgb_symonly, score_xgb_symonly = run_model(X,y)

LIGHTGBM RSME 12.005587292123757
XGBOOST RSME 12.06491995107944


Concat symbolic and train data and rerun model 

In [19]:
X = new_df_trn.drop([target],axis =1)
y= new_df_trn[target]

X = sym_trn
y= df_trn[target]
score_lgb_sym_all, score_xgb_sym_all = run_model(X,y)

LIGHTGBM RSME 12.005587292123757
XGBOOST RSME 12.06491995107944


In [20]:
print("LIGHTGBM\n")
print("Baseline:", score_lgb_base)
print("Manual Features:", score_lgb_manual)
print("Symbolic Features:", score_lgb_sym_all)

LIGHTGBM

Baseline: 11.432613403660998
Manual Features: 11.473143800961294
Symbolic Features: 12.005587292123757


In [21]:
print("XGBOOST\n")
print("Baseline:", score_xgb_base)
print("Manual Features:", score_xgb_manual)
print("Symbolic Features:", score_xgb_sym_all)

XGBOOST

Baseline: 11.508487774570057
Manual Features: 11.608912835769674
Symbolic Features: 12.06491995107944


# Save Data 

In [22]:
new_df_trn.to_csv(f"all_features_trn_{score_lgb_sym_all:.2f}.csv")
new_df_tst.to_csv(f"all_features_tst_{score_lgb_sym_all:.2f}.csv")

In [23]:
sym_trn.to_csv(f"Symbolic_Feats_trn_{score_lgb_symonly:.2f}.csv")
sym_tst.to_csv(f"Symbolic_Feats_tst_{score_lgb_symonly:.2f}.csv")

# Custom Metric (RMSE)

In [24]:
X = df_train.drop([target],axis =1)
y= df_train[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [25]:
from gplearn.fitness import make_fitness

def _rmse(y, y_pred, w):
    rmse = mean_squared_error(y, y_pred)**0.5
    return rmse

rmse = make_fitness(function=_rmse, greater_is_better=False)

In [26]:
function_set = ['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                'abs', 'neg', 'inv', 'max', 'min']
gp_custom = SymbolicTransformer(generations=40, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0,
                         metric=rmse , feature_names= X.columns) 
gp_custom.fit(X_train, y_train)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.54      1.91025e+08        6          16.4079          16.4079      1.40m
   1    10.48           141695        6          16.4079          16.4079      1.43m
   2    10.66           277320        7           15.315           15.315      1.44m
   3    11.52          531.131        7           15.315           15.315      1.38m
   4    10.47          86910.8        8          14.9079          14.9079      1.31m
   5    11.88          124.855       10          14.8493          14.8493      1.34m
   6    13.82          177.367       10          14.8493          14.8493      1.28m
   7    15.16          197.597       53          14.4079          14.4079      1.27m
   8    14.85          93.0658       31          14.0935          14.0935  

SymbolicTransformer(feature_names=Index(['CementComponent', 'BlastFurnaceSlag', 'FlyAshComponent',
       'WaterComponent', 'SuperplasticizerComponent',
       'CoarseAggregateComponent', 'FineAggregateComponent', 'AgeInDays',
       'is_generated'],
      dtype='object'),
                    function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    generations=40, max_samples=0.9,
                    metric=<gplearn.fitness._Fitness object at 0x7f7d0c4caf10>,
                    parsimony_coefficient=0.0005, population_size=2000,
                    random_state=0, verbose=1)

In [27]:
for function in gp_custom:
    print ("Functions:", function)
    print(f"RMSE: {function.raw_fitness_:.3f} \n")

Functions: add(add(sqrt(inv(min(0.029, mul(mul(log(log(min(log(AgeInDays), FineAggregateComponent))), WaterComponent), WaterComponent)))), add(log(AgeInDays), add(abs(add(add(abs(add(SuperplasticizerComponent, add(log(AgeInDays), log(max(CementComponent, add(SuperplasticizerComponent, add(log(log(max(CementComponent, CoarseAggregateComponent))), sqrt(inv(sqrt(inv(min(0.029, mul(log(log(log(AgeInDays))), WaterComponent))))))))))))), log(AgeInDays)), log(AgeInDays))), log(AgeInDays)))), log(max(CementComponent, log(AgeInDays))))
RMSE: 12.604 

Functions: add(add(sqrt(inv(min(0.029, mul(log(inv(min(0.029, mul(log(log(log(AgeInDays))), WaterComponent)))), WaterComponent)))), add(log(AgeInDays), add(abs(add(add(abs(add(SuperplasticizerComponent, add(log(AgeInDays), log(max(CementComponent, add(SuperplasticizerComponent, add(log(log(max(CementComponent, CoarseAggregateComponent))), sqrt(inv(min(0.029, abs(AgeInDays))))))))))), log(AgeInDays)), log(AgeInDays))), log(AgeInDays)))), log(max(Cem

## Transform and Train model (again)
Transform data and join symbolic dataset to Train and Test

In [28]:
sym_trn_custom = gp_custom.transform(X)
sym_tst_custom = gp_custom.transform(df_test)

#create dataframe
sym_trn_custom = pd.DataFrame(sym_trn_custom, columns = [f"sym_{col}" for col in range(sym_tst.shape[1])])
sym_tst_custom = pd.DataFrame(sym_tst_custom, columns = [f"sym_{col}" for col in range(sym_tst.shape[1])])

#join to train and test
new_df_custom_trn = pd.concat([df_trn.reset_index(drop = True),sym_trn_custom],axis =1)
new_df_custom_tst= pd.concat([df_tst.reset_index(drop = True),sym_tst_custom],axis =1)
new_df_custom_trn

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength,is_generated,...,sym_0,sym_1,sym_2,sym_3,sym_4,sym_5,sym_6,sym_7,sym_8,sym_9
0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38,1,...,18.023355,18.049580,14.693486,18.053756,14.731828,14.719517,14.737485,14.737485,14.737485,14.737485
1,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52,1,...,40.492529,40.492529,40.463868,40.538080,40.463868,40.463868,40.463868,40.463868,40.463868,40.463868
2,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96,1,...,33.899693,33.899693,33.300029,33.922491,33.300029,33.300029,33.300029,33.300029,33.300029,33.300029
3,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05,1,...,46.839359,46.839359,48.765678,46.877440,48.765678,48.765678,48.765678,48.765678,48.765678,48.765678
4,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19,1,...,38.517357,38.517357,39.581545,38.573097,39.581545,39.581545,39.581545,39.581545,39.581545,39.581545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6432,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28,0,...,42.710538,42.710538,42.147554,42.734363,42.147554,42.147554,42.147554,42.147554,42.147554,42.147554
6433,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18,0,...,44.517184,44.517184,43.827785,44.537657,43.827785,43.827785,43.827785,43.827785,43.827785,43.827785
6434,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70,0,...,38.668009,38.668009,38.609184,38.711910,38.609184,38.609184,38.609184,38.609184,38.609184,38.609184
6435,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77,0,...,44.005905,44.005905,43.891826,44.046940,43.891826,43.891826,43.891826,43.891826,43.891826,43.891826


Run model

In [29]:
X = new_df_custom_trn.drop([target],axis =1)
y= new_df_custom_trn[target]
score_custom_lgb_all, score_custom_xgb_all = run_model(X,y)

LIGHTGBM RSME 11.503610011630874
XGBOOST RSME 11.67357968498986


In [30]:
print("LIGHTGBM\n")
print("Baseline:", score_lgb_base)
print("Symbolic Features:", score_lgb_symonly)
print("Custom Metric:", score_custom_lgb_all)

LIGHTGBM

Baseline: 11.432613403660998
Symbolic Features: 12.005587292123757
Custom Metric: 11.503610011630874


In [31]:
print("XGBOOST\n")
print("Baseline:", score_xgb_base)
print("Symbolic Features:", score_xgb_symonly)
print("Custom Metric:", score_custom_xgb_all)

XGBOOST

Baseline: 11.508487774570057
Symbolic Features: 12.06491995107944
Custom Metric: 11.67357968498986


# Save

In [32]:
new_df_custom_trn.to_csv(f"all_features_trn_{score_custom_lgb_all:.2f}.csv")
new_df_custom_tst.to_csv(f"all_features_tst_{score_custom_lgb_all:.2f}.csv")

sym_trn_custom.to_csv(f"Symbolic_Custom_trn_{score_custom_lgb_all:.2f}.csv")
sym_tst_custom.to_csv(f"Symbolic_Custom_tst_{score_custom_lgb_all:.2f}.csv")